In [21]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from pymongo import MongoClient
from tqdm import tqdm
import time
import random
from datetime import datetime
import copy
import re

In [22]:
client = MongoClient(host="", port=, username='', password='')
db = client['BOF2024_crawling']
collection = db['weibo_crawling']
data_list = list(collection.find({"검색키워드" : "부산원아시아페스티벌"}, {"_id": 1, "URL": 1}))

In [23]:
data_list1 = copy.deepcopy(data_list)
driver = webdriver.Chrome()
driver.maximize_window()

In [24]:
# Weibo 로그인 페이지로 이동
weibo_login_url = 'https://weibo.com/login.php'
driver.get(weibo_login_url)

# 로그인 대기 (로그인 정보 직접 입력)
WebDriverWait(driver, 20).until(EC.url_changes(weibo_login_url))  # 로그인 완료될 때까지 대기 (필요 시 시간 조정)

# 로그인 후 쿠키 저장
cookies = driver.get_cookies()

In [25]:
def process_url(url):
    try:
        driver.get(url)
        
        # 쿠키 추가
        for cookie in cookies:
            driver.add_cookie(cookie)
        
        # 페이지 새로고침하여 쿠키 적용
        driver.refresh()
        
        # 페이지가 완전히 로드될 때까지 대기
        time.sleep(10)  # 페이지 로드 시간 충분히 주기 (필요에 따라 조정)

        def scroll_to_load_comments():
            SCROLL_PAUSE_TIME = 1.5
            last_height = driver.execute_script("return document.documentElement.scrollHeight")
            while True:
                driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
                time.sleep(SCROLL_PAUSE_TIME)
                new_height = driver.execute_script("return document.documentElement.scrollHeight;")
                if new_height == last_height:
                    break
                last_height = new_height
            time.sleep(5)

        def load_all_comments():
            while True:
                scroll_to_load_comments()
                try:
                    more_content_button = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, '.Bottom_text_1kFLe.Bottom_highlight_2bf0X'))
                    )
                    more_content_button.click()
                    time.sleep(5)  # 버튼 클릭 후 로딩 시간 대기
                except TimeoutException:
                    print("더보기 버튼을 찾지 못했습니다. 더 이상 댓글이 없을 수 있습니다.")
                    break

        load_all_comments()
        
        # 사용자 이름과 댓글 내용 가져오기
        item_elements = driver.find_elements(By.CSS_SELECTOR, '.vue-recycle-scroller__item-view')  # 아이템 요소 찾기

        # 결과 저장용 리스트
        comments = []

        # 각 아이템 요소 순회
        for index, item in enumerate(item_elements):
            try:
                # 사용자 이름 요소 찾기
                username_element = WebDriverWait(item, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, '.text a[usercard]'))
                    )
                username = username_element.text.strip() if username_element else 'N/A'
                
                # 댓글 내용 요소 찾기
                comment_elements = item.find_elements(By.CSS_SELECTOR, '.text span')
                # 모든 span 요소의 텍스트를 결합합니다.
                comment = ' '.join([comment.text.strip() for comment in comment_elements if comment.text.strip()])
                
                # 댓글 작성일자 가져오기
                info_element = item.find_element(By.CSS_SELECTOR, '.info')
                date_text = info_element.text.strip()

                # 작성 일자 변환
                try:
                    # 작성 일자에서 패턴 추출 (23-11-4 14:48 형식)
                    match = re.search(r'(\d{1,2})-(\d{1,2})-(\d{1,2}) (\d{1,2}:\d{2})', date_text)
                    if match:
                        year = match.group(1)
                        month = match.group(2).zfill(2)  # 월을 두 자리로 만듦
                        day = match.group(3).zfill(2)    # 일을 두 자리로 만듦
                        formatted_date = f"20{year}.{month}.{day}."  # 연도와 포맷 조정
                    else:
                        formatted_date = "Unknown"
                except Exception as e:
                    formatted_date = "Unknown"
                    print(f"Error while formatting date: {e}")

                # 출력 포맷 변경 (24-6-12 형식을 2024.06.12. 형식으로 변경)
                if formatted_date != "Unknown":
                    formatted_date = formatted_date.replace('-', '.')

                # 좋아요 개수 가져오기
                try:
                    like_count_elem = item.find_element(By.CSS_SELECTOR, '.woo-like-count')
                    like_count = int(like_count_elem.text.strip())
                except NoSuchElementException:
                    like_count = 0

                # 결과 저장
                if username and comment:  # 사용자 이름과 댓글 내용이 모두 있는 경우만 저장
                    comments.append({
                        "사용자 이름": username,
                        "댓글 내용": comment,
                        "작성일자": formatted_date,
                        "댓글 좋아요": like_count
                    })

            # 결과 출력
                    print(f"사용자 이름: {username}, 댓글 내용: {comment}, 작성일자: {formatted_date}, 좋아요: {like_count}")
            
            except Exception as e:
                print(f"Error occurred: {e}")

        # 총 댓글 수 출력
        print(f"총 댓글 수: {len(comments)}")

    except Exception as e:
        print(f"URL 처리 중 오류 발생: {url}, 오류 내용: {e}")

    return comments

In [ ]:
# URL을 하나씩 열기 및 tqdm 사용
for index, data in enumerate(tqdm(data_list, desc="URL 처리 진행 중", unit="URL")):
    url = data['URL']
    comments = process_url(url)
    
    if comments is not None:
        comment_count = len(comments)  # 댓글 개수 계산

        # MongoDB에 데이터 삽입
        try:
            collection.update_one(
                {"_id": data["_id"]},
                {"$set": {"댓글": comments, "댓글 개수": comment_count}},
                upsert=False
            )
            print(f"{url}의 댓글 데이터 적재 완료")
        except Exception as e:
            print(f"{url}의 댓글 데이터 적재 중 오류 발생: {e}")

    # 각 요청 사이에 랜덤 대기 시간 추가 (5~10초 사이)
    time.sleep(random.randint(5, 10))

In [ ]:
driver.quit()